In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

In [ ]:
Images_Weight = 128
Images_Height = 128
Image_size = (Images_Weight, Images_Height)
Image_Channels=3

In [ ]:
filenames = "../input/dogs-vs-cats/train.zip"

categories = []
for f_name in filenames:
  category = f_name.split('.')[0]
  if categories == 'dog':
    categories.append(1)
  else:
    categories.append(0)

df = pd.DataFrame({
    'filename' : filenames,
    'category' : categories
})

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,\
     Dropout, Flatten,Dense,Activation,\
     BatchNormalization

model = Sequential()
model.add(Conv2D(32,
                 (3,3), activation='relu',input_shape=
                 (Images_Weight, Images_Height, Image_Channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))

model.compile(loss='categorical_crossentropy',
  optimizer='rmsprop',metrics=['accuracy'])

model.summary()

In [ ]:
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystopping = EarlyStopping(patience = 10)
learning_rate = ReduceLROnPlateau(monitor = "val_acc" , verbose = 1, patience =2, factor = 0.5, min_lr = 0.00001)
callbacks = [earlystopping, learning_rate]

In [ ]:
df["category"] = df["category"].replace({'cat':0, 'dog':1})
train_df,validate_df = train_test_split(df,test_size=0.20,
  random_state=42)

train_df=train_df.reset_index(drop=True)
validate_df=validate_df.reset_index(drop=True)

total_train = train_df.shape[0]
validate_train = validate_df.shape[0]
batch_size = 15

Training and valdiation Data generator

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=15, rescale=1./255, shear_range=0.1, zoom_range=0.2, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True )
train_generator = train_datagen.flow_from_dataframe(train_df, "../input/dogs-vs-cats/train.zip", x_col = 'filename', y_col = 'category', target_size = Image_size, class_model = 'categorical', batch_size = batch_size)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(validate_df, "../input/dogs-vs-cats/train.zip", x_col = 'filename', y_col = 'category', target_size = Image_size, class_mode = 'categorical', batch_size = batch_size)

test_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1)

test_generator = train_datagen.flow_from_dataframe(train_df, "../input/dogs-vs-cats/test1.zip", x_col = 'filename', y_col ='category', target_size = Image_size, class_mode = 'categorical', batch_size = batcy_size)

Model training

In [ ]:
epochs = 10
history = model.fit_generator(train_generator, epochs = epochs, validation_data = validation_generator, validation_steps = total_validate//batch_size, steps_per_epoch = total_train//batch_size, callbacks = callbacks)